In [ ]:
import os
import numpy as np
import json
import neuroglancer
from matplotlib import pyplot as plt

viewer = neuroglancer.Viewer()
viewer

## MD589 reference

In [ ]:
MD589_VOLUME_PATH = '/net/birdstore/Active_Atlas_Data/data_root/CSHL_volumes/MD589/MD589_full_filled.npy'
MD589 = np.load(MD589_VOLUME_PATH)
MD589 = np.swapaxes(MD589, 0, 2)

In [ ]:
layer = neuroglancer.SegmentationLayer(
    source=neuroglancer.LocalVolume(
        data=MD589, 
        dimensions=neuroglancer.CoordinateSpace(names=['x', 'y', 'z'], units='um', scales=[5, 5, 20]), 
        voxel_offset=(0, 0, 0)
    ),
)

with viewer.txn() as s:
#     s.layers.clear()
    s.layers['MD589'] = layer

## Average Volumes

In [2]:
color_filepath = os.path.join('/home/eddyod/programming/pipeline_utility/neuroglancer/contours/json_cache', 'struct_to_color_2.json')
with open(color_filepath, 'r') as json_file:
    colors = json.load(json_file)
colors = {name.upper(): index for name, index in colors.items()}
    
surround = False
VOL_DIR = '/net/birdstore/Active_Atlas_Data/copied_from_S3/mousebrainatlas-data/CSHL_volumes/atlasV7/atlasV7_10.0um_scoreVolume/score_volumes'
files = os.listdir(VOL_DIR)
volume_files = sorted([f for f in files if f.endswith('.npy') and surround == ('surround' in f)])
origin_files = sorted([f for f in files if f.endswith('.txt') and surround == ('surround' in f)])
    
structure_volume_origin = {}
for volume_filename, origin_filename in zip(volume_files, origin_files):
    prefix = os.path.splitext(volume_filename)[0]
    structure = prefix.replace('atlasV7_10.0um_scoreVolume_', '').replace('_surround_200um', '')
    if structure not in origin_filename:
        print(structure, origin_filename)
        break
    
    try:
        color = colors[structure.upper()]
    except:
        sided = '{}_R'.format(structure.upper())
        color = colors[sided]

    volume = np.load(os.path.join(VOL_DIR, volume_filename))
    origin = np.loadtxt(os.path.join(VOL_DIR, origin_filename))
    
    volume = np.rot90(volume, axes=(0,1))
    volume = np.flip(volume, axis=0)
    volume[volume > 0.8] = color
    volume = volume.astype(np.uint8)
    
    structure_volume_origin[structure] = (volume, origin)
print(structure_volume_origin.keys())

dict_keys(['10N_L', '10N_R', '12N', '3N_L', '3N_R', '4N_L', '4N_R', '5N_L', '5N_R', '6N_L', '6N_R', '7N_L', '7N_R', '7n_L', '7n_R', 'AP', 'Amb_L', 'Amb_R', 'DC_L', 'DC_R', 'IC', 'LC_L', 'LC_R', 'LRt_L', 'LRt_R', 'PBG_L', 'PBG_R', 'Pn_L', 'Pn_R', 'RMC_L', 'RMC_R', 'RtTg', 'SC', 'SNC_L', 'SNC_R', 'SNR_L', 'SNR_R', 'Sp5C_L', 'Sp5C_R', 'Sp5I_L', 'Sp5I_R', 'Sp5O_L', 'Sp5O_R', 'Tz_L', 'Tz_R', 'VCA_L', 'VCA_R', 'VCP_L', 'VCP_R', 'VLL_L', 'VLL_R'])


## Adding volumes individually to NG
### (Origin of each volume is set in NG)

In [3]:
with viewer.txn() as s:
    s.layers.clear()

for structure, (volume, origin) in structure_volume_origin.items():
#     if structure not in ['SC', 'IC', 'SNR_L', 'SNR_R', 'Pn_L', 'Pn_R', 'SNC_L', 'SNC_R']:
#         continue
        
    volume_layer = neuroglancer.SegmentationLayer(
        source = neuroglancer.LocalVolume(
            data=volume, 
            dimensions=neuroglancer.CoordinateSpace(names=['x', 'y', 'z'], units='um', scales=[10, 10, 10]), 
            voxel_offset=origin),
    )
    with viewer.txn() as s:
        s.layers[structure] = volume_layer

## Aggregate volumes in one volume and add to NG 
### (Origin of each volume is set in numpy)

In [7]:
x_length = 2000
y_length = 1500
z_length = 1500
full_brain_volume_annotated = np.zeros((x_length, y_length, z_length), dtype=np.uint8)

for structure, (volume, origin) in structure_volume_origin.items():
#     if structure not in ['SC', 'IC', 'SNR_L', 'SNR_R', 'Pn_L', 'Pn_R', 'SNC_L', 'SNC_R']:
#         continue    
    
    x, y, z = origin
    x_start = int(x) + x_length // 2
    y_start = int(y) + y_length // 2
    z_start = int(z) + z_length // 2
    x_end = x_start + volume.shape[0]
    y_end = y_start + volume.shape[1]
    z_end = z_start + volume.shape[2]

    full_brain_volume_annotated[x_start:x_end, y_start:y_end,z_start:z_end] += volume


all_volume_layer = neuroglancer.SegmentationLayer(
    source = neuroglancer.LocalVolume(
        data=full_brain_volume_annotated, 
        dimensions=neuroglancer.CoordinateSpace(names=['x', 'y', 'z'], units='um', scales=[10, 10, 10])
        , 
        voxel_offset=(0, 0, 0)
    ),
)

with viewer.txn() as s:
    s.layers.clear()
    s.layers['all'] = all_volume_layer

In [8]:
print(viewer)

http://127.0.0.1:39617/v/ac0c55d52a6be014ef8eff830de9cc85feac528d/
